In [38]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from pandas import Series,DataFrame

import warnings
warnings.filterwarnings('ignore')

import scipy.stats as stats
import statsmodels.regression.linear_model as lmr
import statsmodels.discrete.discrete_model as sm
from statsmodels.api import add_constant

from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics
from collections import Counter
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.metrics import r2_score
from statsmodels.formula.api import ols

%matplotlib inline

In [39]:
pisaTrain = pd.read_csv('pisa2009train.csv')
pisaTest = pd.read_csv('pisa2009test.csv')
print('No. of students:',pisaTrain.shape[0])

No. of students: 3663


In [40]:
f_avg = pisaTrain.groupby('male').readingScore.mean()[0]
m_avg = pisaTrain.groupby('male').readingScore.mean()[1]  # 1 for Male as per the column description in file

print('Average reading test score of Females:',round(f_avg,2))
print('Average reading test score of Males:',round(m_avg,2))

Average reading test score of Females: 512.94
Average reading test score of Males: 483.53


In [55]:
null_columns = DataFrame(pisaTrain.isnull().sum())

print('Below are the columns with null values in training set..\n')
for i,j in  null_columns.iterrows():
    if(null_columns[0][i]!=0):
        print(i)

Below are the columns with null values in training set..

raceeth
preschool
expectBachelors
motherHS
motherBachelors
motherWork
fatherHS
fatherBachelors
fatherWork
selfBornUS
motherBornUS
fatherBornUS
englishAtHome
computerForSchoolwork
read30MinsADay
minutesPerWeekEnglish
studentsInEnglish
schoolHasLibrary
schoolSize


In [53]:
null_columns

,0
grade,0
male,0
raceeth,35
preschool,56
expectBachelors,62
motherHS,97
motherBachelors,397
motherWork,93
fatherHS,245
fatherBachelors,569


In [6]:
pisaTrain = pisaTrain.dropna(axis=0)
pisaTest = pisaTest.dropna(axis=0)

print('No. of observations in training set after removing null:',pisaTrain.shape[0])
print('No. of observations in testing set after removing null:',pisaTest.shape[0])

No. of observations in training set after removing null: 2414
No. of observations in testing set after removing null: 990


In [7]:
Dtype = DataFrame(pisaTrain.dtypes,columns=['DType'])
Unique = DataFrame(pisaTrain.nunique(),columns=['Unique'])

varCat = pd.concat([Dtype,Unique],axis=1)
varCat = varCat[varCat.DType == 'object']
varCat = varCat[varCat.Unique > 3]

print('Categorical column(s) with more than 3 levels are..\n')
for i,j in  varCat.iterrows():
    print(i)

Categorical column(s) with more than 3 levels are..

raceeth


In [58]:
df_raceeth = pd.get_dummies(pisaTrain['raceeth'])

#No need to indicate NaNs as we have already dropped it. (dummy_na property)
df_raceeth.drop(labels=['White'],axis=1,inplace=True)
#We are dropping White because it is the reference

df_raceeth.head()

#Each observation can have only one raceeth value.
#Hence, if all the values are zero for example the first row below White = 1 (i.e. raceeth is White)
#Similarly, if any one value is a non-zero then the raceeth is the respective attribute.

,American Indian/Alaska Native,Asian,Black,Hispanic,More than one race,Native Hawaiian/Other Pacific Islander
0,0,0,0,0,0,0
1,0,0,0,0,0,0
2,0,0,0,0,0,0
3,0,0,1,0,0,0
4,0,0,0,1,0,0


In [79]:
#Therefore, If we select White as the reference level,the values below will be added to the Regression Model.

print('Binary variables that will be included in the regression model are..\n')
for i,j in  DataFrame(df_raceeth.columns).iterrows(): #for index, row in df.iterrows()
    print(DataFrame(df_raceeth.columns)[0][i])

Binary variables that will be included in the regression model are..

American Indian/Alaska Native
Asian
Black
Hispanic
More than one race
Native Hawaiian/Other Pacific Islander


In [83]:
df_raceeth.columns[3]

'Hispanic'

In [10]:
df_Asian = df_raceeth[df_raceeth['Asian']==1]
listColumns = list(df_Asian.columns)

print('Columns that would be set to 0 for an Asian student..\n')
for i in listColumns:
    if(df_Asian[i].all()==0):
        print(i)
#The refernce is white and hence,
print('White')

Columns that would be set to 0 for an Asian student..

American Indian/Alaska Native
Black
Hispanic
More than one race
Native Hawaiian/Other Pacific Islander
White


In [86]:
listColumns

['American Indian/Alaska Native',
 'Asian',
 'Black',
 'Hispanic',
 'More than one race',
 'Native Hawaiian/Other Pacific Islander']

In [11]:
df_Asian = df_raceeth[df_raceeth['Asian']==1]
listColumns = list(df_Asian.columns)

print('Columns that would be set to 0 for a White student..\n')
for i in listColumns:
    print(i)
    
    
#We are printing all the columns because "White" is reference.

Columns that would be set to 0 for a White student..

American Indian/Alaska Native
Asian
Black
Hispanic
More than one race
Native Hawaiian/Other Pacific Islander


In [12]:
df_train = pd.get_dummies(pisaTrain,drop_first=True)
#drop_first=True  ->  American Indian/Alaska Native is the reference

#Dependant and Independant variables
X=df_train.drop(labels=['readingScore'],axis=1)
Y=df_train.readingScore

#Model Creation
lmScore=LinearRegression()
#We need not split X,Y as train and test as our model is going to have every column and the data for testing is given.
lmScore.fit(X,Y)
y_pred = lmScore.predict(X)

#R2 Score
print('R-squared value of lmScore(LinearRegression model) on the training set is',round(r2_score(Y,y_pred),1))

R-squared value of lmScore(LinearRegression model) on the training set is 0.3


In [13]:
rmse=round(np.sqrt(metrics.mean_squared_error(Y,y_pred)),2)
print('Root-Mean Squared Error (RMSE) of lmScore on the training set is',rmse)

Root-Mean Squared Error (RMSE) of lmScore on the training set is 73.37


In [93]:
df_train.head()

,grade,male,preschool,expectBachelors,motherHS,motherBachelors,motherWork,fatherHS,fatherBachelors,fatherWork,...,publicSchool,urban,schoolSize,readingScore,raceeth_Asian,raceeth_Black,raceeth_Hispanic,raceeth_More than one race,raceeth_Native Hawaiian/Other Pacific Islander,raceeth_White
1,11,1,0.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,...,1,0,1173.0,575.01,0,0,0,0,0,1
3,10,0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,...,1,1,2640.0,458.11,0,1,0,0,0,0
4,10,1,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,...,1,1,1095.0,613.89,0,0,1,0,0,0
7,10,0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,...,1,0,1913.0,439.36,0,0,0,0,0,1
9,10,1,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,...,1,0,899.0,465.90,0,0,0,1,0,0


In [14]:
df_train.iloc[1,:].values

array([1.0000e+01, 0.0000e+00, 1.0000e+00, 1.0000e+00, 0.0000e+00,
       0.0000e+00, 1.0000e+00, 1.0000e+00, 0.0000e+00, 1.0000e+00,
       1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00,
       1.0000e+00, 2.0000e+02, 2.3000e+01, 1.0000e+00, 1.0000e+00,
       1.0000e+00, 2.6400e+03, 4.5811e+02, 0.0000e+00, 1.0000e+00,
       0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00])

In [99]:
df_train.iloc[19,:].values

array([1.0000e+01, 1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00,
       1.0000e+00, 0.0000e+00, 1.0000e+00, 0.0000e+00, 1.0000e+00,
       1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00,
       1.0000e+00, 2.5000e+02, 2.5000e+01, 1.0000e+00, 1.0000e+00,
       0.0000e+00, 1.9130e+03, 6.4965e+02, 0.0000e+00, 0.0000e+00,
       0.0000e+00, 0.0000e+00, 0.0000e+00, 1.0000e+00])

In [15]:
#Random Values from our training dataset with values changed for Grade

#A - 1.1000e+01 as Grade value(first value)
A = DataFrame([[1.1000e+01, 0.0000e+00, 1.0000e+00, 1.0000e+00, 0.0000e+00,
       0.0000e+00, 1.0000e+00, 1.0000e+00, 0.0000e+00, 1.0000e+00,
       1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00,
       1.0000e+00, 2.0000e+02, 2.3000e+01, 1.0000e+00, 1.0000e+00,
       1.0000e+00, 2.6400e+03, 0.0000e+00, 1.0000e+00,
       0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00]])

#B - 0.9000e+01 as Grade value(first value)
B =  DataFrame([[0.9000e+01, 0.0000e+00, 1.0000e+00, 1.0000e+00, 0.0000e+00,
       0.0000e+00, 1.0000e+00, 1.0000e+00, 0.0000e+00, 1.0000e+00,
       1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00,
       1.0000e+00, 2.0000e+02, 2.3000e+01, 1.0000e+00, 1.0000e+00,
       1.0000e+00, 2.6400e+03, 0.0000e+00, 1.0000e+00,
       0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00]])

In [16]:
#Prediction

y_pred_A = lmScore.predict(A)
y_pred_B = lmScore.predict(B)
print('Predicted reading score of student A minus the predicted reading score of student B is',round(list(y_pred_A-y_pred_B)[0],2))


Predicted reading score of student A minus the predicted reading score of student B is 59.09


In [17]:
listofcolumns = list(X.columns)
j=0
for i in listofcolumns:
    if(i=='raceeth_Asian'):
        print('Coefficient of raceethAsian is',lmScore.coef_[j])
    j=j+1

Coefficient of raceethAsian is 63.16700194599064


In [18]:
#Considering Multicollinearity

vif=DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
vif['Columns']=X.columns
vif

,VIF Factor,Columns
0,146.403518,grade
1,2.179933,male
2,3.909335,preschool
3,6.815514,expectBachelors
4,14.637315,motherHS
5,2.403052,motherBachelors
6,4.039440,motherWork
7,12.052780,fatherHS
8,2.463370,fatherBachelors
9,7.269092,fatherWork


In [19]:
#Based on VIF, we can drop the columns and create X1 as below.(Assuming a threshold as 8 for VIF)

X1=X.iloc[:,[1,2,3,5,6,8,9,15,16,20,21,22,23,24,25,26,27]]
X1.head()

,male,preschool,expectBachelors,motherBachelors,motherWork,fatherBachelors,fatherWork,read30MinsADay,minutesPerWeekEnglish,urban,schoolSize,raceeth_Asian,raceeth_Black,raceeth_Hispanic,raceeth_More than one race,raceeth_Native Hawaiian/Other Pacific Islander,raceeth_White
1,1,0.0,0.0,1.0,1.0,0.0,1.0,1.0,450.0,0,1173.0,0,0,0,0,0,1
3,0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,200.0,1,2640.0,0,1,0,0,0,0
4,1,1.0,0.0,0.0,1.0,0.0,0.0,1.0,250.0,1,1095.0,0,0,1,0,0,0
7,0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,300.0,0,1913.0,0,0,0,0,0,1
9,1,1.0,1.0,1.0,1.0,0.0,1.0,0.0,294.0,0,899.0,0,0,0,1,0,0


In [20]:
#Significance Code
#Based on p-value(Significance Code), we can further drop the columns and create X11 as below.
X11=X1.drop(labels=['motherWork','preschool','minutesPerWeekEnglish','fatherWork','urban','male'],axis=1)

#Columns are selected in a looping process in the same order.

In [21]:
X2 = add_constant(X11)
lm= lmr.OLS(Y,X2)
lm2=lm.fit()
lm2.pvalues
lm2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           readingScore   R-squared:                       0.276
Model:                            OLS   Adj. R-squared:                  0.273
Method:                 Least Squares   F-statistic:                     83.17
Date:                Sun, 20 Jan 2019   Prob (F-statistic):          2.01e-159
Time:                        19:04:29   Log-Likelihood:                -13880.
No. Observations:                2414   AIC:                         2.778e+04
Df Residuals:                    2402   BIC:                         2.785e+04
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
==================================================================================================================
                                                     coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------------------------
const                                            367.7203     17.399     21.135      0.000     333.602     401.839
expectBachelors                                   64.4453      4.308     14.958      0.000      55.997      72.894
motherBachelors                                   13.3698      3.894      3.434      0.001       5.734      21.005
fatherBachelors                                   20.6359      4.010      5.146      0.000      12.772      28.500
read30MinsADay                                    38.7125      3.415     11.336      0.000      32.016      45.409
schoolSize                                         0.0061      0.002      3.209      0.001       0.002       0.010
raceeth_Asian                                     82.7548     18.842      4.392      0.000      45.806     119.704
raceeth_Black                                     10.6133     17.814      0.596      0.551     -24.319      45.546
raceeth_Hispanic                                  39.6530     17.478      2.269      0.023       5.379      73.927
raceeth_More than one race                        63.1154     19.051      3.313      0.001      25.758     100.473
raceeth_Native Hawaiian/Other Pacific Islander    83.3619     24.138      3.454      0.001      36.029     130.695
raceeth_White                                     80.5286     17.199      4.682      0.000      46.803     114.255
==============================================================================
Omnibus:                       11.397   Durbin-Watson:                   2.016
Prob(Omnibus):                  0.003   Jarque-Bera (JB):               11.485
Skew:                          -0.160   Prob(JB):                      0.00321
Kurtosis:                       2.894   Cond. No.                     4.78e+04
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 4.78e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [22]:
print('Columns that can be removed as part of significance codes are below..\n')
for i in df_train.columns:
    if i not in X11.columns:
        print(i)

Columns that can be removed as part of significance codes are below..

grade
male
preschool
motherHS
motherWork
fatherHS
fatherWork
selfBornUS
motherBornUS
fatherBornUS
englishAtHome
computerForSchoolwork
minutesPerWeekEnglish
studentsInEnglish
schoolHasLibrary
publicSchool
urban
readingScore


In [23]:
df_test = pd.get_dummies(pisaTest,drop_first=True)
#No null values in pisaTest to handle
#drop_first=True  ->  American Indian/Alaska Native is the reference

#Dependant and Independant variables
X=df_test.drop(labels=['readingScore'],axis=1)
Y=df_test.readingScore

predTest=lmScore.predict(X)
print('Difference between min and max test scores on testing set is',round(max(predTest)-min(predTest)))

Difference between min and max test scores on testing set is 284.0


In [32]:
sse=(round(metrics.mean_squared_error(Y,predTest)))*X.shape[0]
print('Sum of Squared Error (SSE) of lmScore on the training set is',sse)
rmse=round(np.sqrt(metrics.mean_squared_error(Y,predTest)),2)
print('Root-Mean Squared Error (RMSE) of lmScore on the training set is',rmse)

Sum of Squared Error (SSE) of lmScore on the training set is 5761800.0
Root-Mean Squared Error (RMSE) of lmScore on the training set is 76.29


In [25]:
avg = pisaTrain.readingScore.mean()
print('By Baseline model, prediction score is,',avg)

By Baseline model, prediction score is, 517.9628873239436


In [26]:
l_avg=list()
for i in Y:
    l_avg.append(avg)
sse=(round(metrics.mean_squared_error(Y,l_avg)))*X.shape[0]
print('Sum of Squared Error (SSE) of lmScore on the training set is',sse)

Sum of Squared Error (SSE) of lmScore on the training set is 7802190.0


In [27]:
print('R-squared value of lmScore(LinearRegression model) on the training set is',round(r2_score(Y,predTest),2))

R-squared value of lmScore(LinearRegression model) on the training set is 0.26
